<a href="https://colab.research.google.com/github/sv2639/proj3/blob/main/proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem 2 Implementing a Meet-in-the-Middle Attack on a Mini Block Cipher

### Project Overview:

This project involves students in the implementation and analysis of a meet-in-the-middle (MITM) attack against a simplified block cipher called mini block cipher based on the idea of the SAES (FYI, it is better to
use the simplified DES). The primary goal is to provide hands-on experience with the MITM attack, showcasing its effectiveness against certain cryptographic algorithms and understanding why modern ciphers like AES are designed to be immune to such attacks.

MITM or Meet-in-the-Middle Attack” is an exhaustive key search attack1. It is a cryptanalytic attack applicable to ciphers based on composition of multiple rounds of substitutions and permutations. It works by finding plaintext-ciphertext pairs that map to the same intermediate value after partial encryption/decryption2.

SAES starts with the key expansion, then works on encryption to get ciphertext and on decryption to recover the plaintext. The key expansion generates three keys. The first key, Key0, is used for the add round key to the plaintext. The second key, Key1, is used to perform Round 1 transformation on state, defined as encrypt_round1(). The third key, Key2, is used to perform Round 2 transformations on state,
defined as encrypt_round2(). For decryption, it is reverse. Key2 is used to perform inverse Round 2 transformations on ciphertext, defined as decrypt_round2(). Key1 is used to perform inverse Round 1
transformations on state, defined as decrypt_round1(). So, the pseudo-code for SAES would be the following. It is assumed the key size is 16 bit.

1. Get (Key0, Key1, Key2) from Key K using the key expansion.

2. Two rounds of Encrypt to get the ciphertext.
  
        I. AddRoundKey Key0
        
        II. encrypt_round1() using Key1
    
            i. Subsititute()
    
            ii. Shift()
    
            iii. Mix()
    
            iv. AddRoundKey()
        
        III. encrypt_round2() using Key2
    
            i. Subsititute()
    
            ii. Shift()
    
            iii. AddRoundKey()

3. Two rounds of decryption to recover the plaintext
  
        I. decrypt_round2() using Key 2
    
            i. AddRoundKey()
    
            ii. Shift()
    
            iii. Subsititute()
  
        II. decrypt_round1() using Key1

            i. AddRoundKey()
            
            ii. Mix()
            
            iii. Shift()
            
            iv. Subsititute()

        III. AddRoundKey Key0

For the project, we need to modify the pseudo code to fit our class project. The pseudo-code for this
mini block cipher based on the SAES is as follows:

1. Get (Key0, Key1, Key2) from Key K using the key expansion.

2. Two rounds of Encrypt to get the ciphertext.

        I. encrypt_round1() using Key1 on plaintext, P, and get intermediate state X

            i. Subsititute()

            ii. Shift()

            iii. Mix()

            iv. AddRoundKey()

        II. encrypt_round2() using Key2 on intermediate state X, and get the ciphertext C.

            i. Subsititute()
  
            ii. Shift()

            iii. AddRoundKey()

3. Two rounds of decryption to recover the plaintext

        I. decrypt_round2() using Key 2 on the ciphertext, C, and get intermediate state, Y

            i. AddRoundKey()

            ii. Shift()
  
            iii. Subsititute()

        II. decrypt_round1() using Key1 on Y to get plaintext P.

            i. AddRoundKey()

            ii. Mix()

            iii. Shift()

            iv. Subsititute()

The meet in the middle attack strategy to mini block cipher:

      A. Calculate X = encrypt_round1(Key1, P)

      B. Calculate X’ = decrypt_round2(Key2, C).

      C. Find out a pair (Key1, Key2) such at X = X’

      D. For one specific (P, C), there will be many matched pairs, we need to use another plaintext and ciphertext pair to eliminate some of the matched pairs. Ideally, we shall get one matched key pair. This pair can be used to get plaintext from any cyphertext. In other words, we cracked the block cipher.

Project Tasks

Task 1: Implementing Mini Block Cipher with key size 16 bit and block size 16 bit:

      a) Students will implement the Mini Block Cipher encryption and decryption functions (1, 2I, 2II, 3I, 3II) using jupyter notebook.

      b) Make at least ten pairs of plaintexts and ciphertexts.

Task 2: Meet-in-the-Middle Attack Implementation:

      a) Students need to implement the meet in the middle attack strategy to mini block cipher (a-d).

      b) Show key pair(s) that works for the pair of plaintext and ciphertext from task1 (b). Ideally, it should have only one key pair works.

Task 3: Analyze the time and memory complexity of the attack compared with the naive exhaustive key search.

      a) What is the key space for the mini block cipher?

      b) Image the mini block cipher is executed twice to generate a cipher text. It is called double mini cipher block. We need a key in 32 bits. The first 16 to the first mini block cipher, the remaining 16 to the second mini block cipher. The meet in the middle attack is to match the state for the first encryption of mini block cipher and the second decryption mini block. How many operations are needed to such attack?

      c) If we do exhaustive key search for the double mini block cipher, how many operations are needed?

      d) What is the tradeoff for the MITM attack (speed, memory, etc.)?





1 https://en.wikipedia.org/wiki/Meet-in-the-middle_attack

2 For example, https://youtu.be/S-EhbhDXUwM (It is a bit long…)

In [22]:
import random
from collections import defaultdict

# Helper Functions (based on SAES specifications)

def substitute(block):
    # Simple substitution: invert the bits for simplicity
    return ''.join(['1' if b == '0' else '0' for b in block])

def shift(block):
    # Circular shift by 1
    return block[1:] + block[0]

def mix(block):
    # Simple mix function for demonstration (can be expanded)
    constant = "1100"  # A 4-bit constant for mixing
    return ''.join(['1' if b != constant[i] else '0' for i, b in enumerate(block)])

def add_round_key(block, key):
    # XOR the block with the key
    return ''.join(['1' if b != k else '0' for b, k in zip(block, key)])

# Round functions for SAES encryption and decryption
def encrypt_round1(block, key):
    block = substitute(block)
    block = shift(block)
    block = mix(block)
    block = add_round_key(block, key)
    return block

def encrypt_round2(block, key):
    block = substitute(block)
    block = shift(block)
    block = add_round_key(block, key)
    return block

def decrypt_round2(block, key):
    block = add_round_key(block, key)
    block = shift(block)  # Inverse of the shift
    block = substitute(block)  # Inverse of substitution
    return block

def decrypt_round1(block, key):
    block = add_round_key(block, key)
    block = mix(block)  # Inverse of mix
    block = shift(block)  # Inverse of shift
    block = substitute(block)  # Inverse of substitution
    return block

# SAES Encryption and Decryption
def encrypt(plaintext, keys):
    key0, key1, key2 = keys  # Unpack all 3 keys
    ciphertext = ''
    # Process each 4-bit block
    for i in range(0, len(plaintext), 4):
        block = plaintext[i:i+4]
        # First round with Key1
        intermediate = encrypt_round1(block, key1)
        # Second round with Key2
        ciphertext += encrypt_round2(intermediate, key2)
    return ciphertext

def decrypt(ciphertext, keys):
    key0, key1, key2 = keys  # Unpack all 3 keys
    plaintext = ''
    # Process each 4-bit block
    for i in range(0, len(ciphertext), 4):
        block = ciphertext[i:i+4]
        # First round with Key2 (reverse order)
        intermediate = decrypt_round2(block, key2)
        # Second round with Key1 (reverse order)
        plaintext += decrypt_round1(intermediate, key1)
    return plaintext

# Text to Binary Conversion (based on provided mapping)
def text_to_binary(text):
    # Mapping of characters A-P to binary strings
    char_to_bin = {
        'A': '0000', 'B': '0001', 'C': '0010', 'D': '0011',
        'E': '0100', 'F': '0101', 'G': '0110', 'H': '0111',
        'I': '1000', 'J': '1001', 'K': '1010', 'L': '1011',
        'M': '1100', 'N': '1101', 'O': '1110', 'P': '1111'
    }
    return ''.join([char_to_bin[char] for char in text.upper() if char in char_to_bin])

# Binary to Text Conversion
def binary_to_text(binary):
    bin_to_char = {
        '0000': 'A', '0001': 'B', '0010': 'C', '0011': 'D',
        '0100': 'E', '0101': 'F', '0110': 'G', '0111': 'H',
        '1000': 'I', '1001': 'J', '1010': 'K', '1011': 'L',
        '1100': 'M', '1101': 'N', '1110': 'O', '1111': 'P'
    }
    # Divide binary string into chunks of 4 bits
    return ''.join([bin_to_char[binary[i:i+4]] for i in range(0, len(binary), 4)])

# Example of Key Expansion (simplified for demonstration)
def key_expansion(key):
    # We assume a simple key expansion (split the key into three parts)
    key0 = key[:4]  # 4 bits
    key1 = key[4:8]  # 4 bits
    key2 = key[8:12]  # 4 bits (take only the first 4 bits from the third part)
    return (key0, key1, key2)

# Generating Random Keys for Testing
def generate_random_key():
    return ''.join(random.choice('01') for _ in range(12))  # 12-bit key (split into 3 keys)

# Example Test for Task 1 and 2
key = generate_random_key()
key_exp = key_expansion(key)

# Print out the key expansion before using it in encryption
print(f"Key Expansion (key_exp): {key_exp}")

# Create plaintext-ciphertext pairs
plaintext = "FOO"  # Example text
plaintext_binary = text_to_binary(plaintext)  # Convert text to binary

print(f"Original Plaintext: {plaintext}")
print(f"Binary Plaintext: {plaintext_binary}")

ciphertext = encrypt(plaintext_binary, key_exp)

# Print the encryption result
print(f"Ciphertext: {ciphertext}")

# Decrypt the ciphertext and convert back to text
decrypted_binary = decrypt(ciphertext, key_exp)
decrypted_text = binary_to_text(decrypted_binary)

# Print the decryption result
print(f"Decrypted Binary: {decrypted_binary}")
print(f"Decrypted Plaintext: {decrypted_text}")


Key Expansion (key_exp): ('0101', '0110', '0101')
Original Plaintext: FOO
Binary Plaintext: 010111101110
Ciphertext: 010110111011
Decrypted Binary: 010111101110
Decrypted Plaintext: FOO


 In the pseudo code for question 2 in project 3, it says the following:
> For the project, we need to modify the pseudo code to fit our class project. The pseudo-code for this
> mini block cipher based on the SAES is as follows:
> 1. Get (Key0, Key1, Key2) from Key K using the key expansion


It says to get key0, key1, and key2

But then steps 2 and 3 say to use key1 and key2, completely skipping key0

The pseudo-code for SAES says to use key0, before either of the encrypt rounds. Meaning key0 actually gets used.

But I don't see where key0 gets used in the mini block cipherSo that's why I'm messaging you.Why does key0 get made if it's not being used?

Secondly. For the substitute portion of the algorithm are we using the substitution box from the slides of this week's lecture?



SyntaxError: invalid syntax (<ipython-input-13-d633f1496d47>, line 1)